# 刺激系をコントロールするためのプログラム
# Function generatorとDAQをコントロールすることが目的

### Fungene
- Keysight visaというライブラリが必要
- pythonでコントロールするためには[PyVISA](https://pyvisa.readthedocs.io/en/latest/)を使用する
    - visa用コードは下から
    https://nfcorp.co.jp/support/manual/pdf/WF1947_48_RemoteControl_InstructionManual003_Jpn.pdf
    - 基本的には?で問い合わせ(取得)そのままで指定(入力・設定)
    - これだけ入れて無理な場合にはpyusbをpipでインストールする
    
### DAQ
- PyDAQmxというライブラリを使用
    - https://pythonhosted.org/PyDAQmx/
    - https://zone.ni.com/reference/en-XX/help/370471AM-01/cdaqmx/help_file_title/
    - https://github.com/clade/PyDAQmx
- NIDAQmxドライバが必要


### Fungeneの刺激生成はPythonでやったことがないため試してみる

In [1]:
## ライブラリの読み込み
import pyvisa as visa
import sys

In [12]:
resource_manager= visa.ResourceManager()

In [13]:
devise_list= resource_manager.list_resources()
print(devise_list)

('USB0::0x0D4A::0x000D::9201982::INSTR', 'ASRL1::INSTR', 'ASRL3::INSTR', 'PXI0::8-4.0::INSTR')


In [14]:
#接続されたすべてのVISA機器リストを出力
def get_devices():
    devise_list= list(resource_manager.list_resources())
    return devise_list

In [19]:
#resource_managerから特定機器を取得
def open_device(address):
    return resource_manager.open_resource(address)

In [20]:
# 機器情報を取得
def identification(device):
    return device.query("*IDN?")

In [21]:
#接続機器リストを取得
device_list=get_devices()
print(get_devices())

['USB0::0x0D4A::0x000D::9201982::INSTR', 'ASRL1::INSTR', 'ASRL3::INSTR', 'PXI0::8-4.0::INSTR']


In [24]:
#ファンクションジェネレータのデータを取得
fungene_device = open_device(device_list[0])
#取得したdeviceの機器情報を取得
print(identification(fungene_device))

NF Corporation,WF1947,9201982,Ver1.40



In [52]:
#取得したdevice(ファンクションジェネレーターに対して刺激設定を出力できるようにする)
## 刺激モード・刺激パラメータの設定
fungene_device.write(":SOURce1:MODE CONT")
fungene_device.write(":SOURce1:FUNCtion:SHAPe SIN")
fungene_device.write(":SOURce1:FREQuency:CW 1000HZ")
fungene_device.write(":SOURce1:VOLTage:LEVel:IMMediate:AMPLitude 1VPP")
fungene_device.write(":SOURce1:BURSt:TRIGger:NCYCles 1000")
#fungene_device.write(":SOURce1:VOLTage:LEVel:IMMediate:OFFSet 0.0V")
#fungene_device.write(":OUTPut1:POLarity SINusoid, NORMal")




37

In [57]:
#outputをonに設定
fungene_device.write(":OUTPut:STATe 1")

17

In [58]:
#triggerをon→波形生成
fungene_device.write(":TRIG 1")

9

fungene_device.write(":OUTPut:STATe 0")

- 波形生成ができた
- modu,humではDAQからの電圧値を外部振幅変調機能として使用している

### 次はDAQと連動させて動かすことを考える
1. DAQにAnalogOut, AnalogINのチャネルを生成
2. 超音波波形情報をFGに格納
3. DAQからtrigger電圧を印加させて超音波波形を生成
4. AEセンサからの電圧を連続配列としてDAQに入力して格納

In [2]:
# DAQ制御用に必要なライブラリの読み込み
from __future__ import print_function
from ctypes import *
from PyDAQmx import *
import numpy
from matplotlib import pyplot as plt

# メモリ解放用のライブラリ
import gc

### DAQはTaskというオブジェクトを使用して管理するが、Taskは一つしか起動できないらしい
→　そのため1つのTask内で入出力チャンネルの生成→計測を行ってみる

In [4]:
#音刺激用の波形生成
def table(**kwargs):
    keys = list(kwargs.keys())
    values = numpy.array(numpy.meshgrid(*tuple(kwargs.values()))).transpose()
    
    return pandas.DataFrame(values.reshape(numpy.prod(values.shape[:-1]), values.shape[-1]), columns=keys)
#まずは音刺激のデータをそのまま電圧値として入力するところから
def toneburst(sampling_rate, frequency, db, rise_fall, duration):
    ###frequency:各周波数, amplitude:Max電圧 [V], rise_fall:立ち上がり/下り時間 [s], duration: 音刺激の長さ [s] 
    fade = int(rise_fall*sampling_rate)
    #amplitude = calib_pure[(calib_pure.frequency == frequency)&(calib_pure.db == db)].amplitude.values
    amplitude=1.0
    wave = amplitude*numpy.sin(numpy.linspace(0, numpy.pi*2*frequency*duration, int(duration*sampling_rate)))
    wave[:fade] *= numpy.linspace(0, 1, fade)
    wave[-fade:] *= numpy.linspace(1, 0, fade)

    trigger = numpy.zeros(int(sampling_rate*duration)) 
    trigger[0 : int(sampling_rate/10000)] = 5
    
    return numpy.array([wave, trigger])

In [ ]:
# InputOutput用のDAQチャンネルの生成
task = Task()
read = int32()
input_data = numpy.zeros((10000,), dtype=numpy.float64)

try:
    # DAQmx Configure Code
    #analog_input.CreateAIVoltageChan("デバイス名/チャンネル","",DAQmx_Val_Cfg_Default,-10.0,10.0,DAQmx_Val_Volts,None)
    task.CreateAIVoltageChan("Dev2/ai0","",DAQmx_Val_Cfg_Default,-10.0,10.0,DAQmx_Val_Volts,None)
    #analog_input.CfgSampClkTiming("",サンプリングレート,DAQmx_Val_Rising,DAQmx_Val_FiniteSamps,サンプル数)
    task.CfgSampClkTiming("",20000.0,DAQmx_Val_Rising,DAQmx_Val_FiniteSamps,40000)

    #DAQmx Write Code for sending signal
    task.CreateAOVoltageChan("Dev2/ao0", "", -10.0, 10.0, DAQmx_Val_Volts, None)
    task.CreateAOVoltageChan("Dev2/ao1", "", -10.0, 10.0, DAQmx_Val_Volts, None)

    # task.DAQmx Write Code
    #task.WriteAnalogF64(chごとのサンプル数, 0, 10.0, DAQmx_Val_GroupByChannel, 書き込むデータ, None, None)
    task.WriteAnalogF64(data.size//2, 0, 10.0, DAQmx_Val_GroupByChannel, data, None, None)

    # DAQmx Start Code
    analog_input.StartTask()

    # DAQmx Read Code
    analog_input.ReadAnalogF64(40000,10.0,DAQmx_Val_GroupByChannel,input_data,40000,byref(read),None)

except DAQError as err:
    print("DAQmx Error: {}".format(err))

finally:
    if task:
        # task.DAQmx Stop Code
        task.StopTask()
        task.ClearTask()
#print("Acquired %d points"%read.value)
assert read.value==40000
#print(data)

